# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [24]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [25]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [26]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [27]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

112


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist -> 0
- firstName of user -> 1
- gender of user -> 2
- item number in session -> 3
- last name of user -> 4
- length of the song -> 5
- level (paid or free song) -> 6
- location of the user -> 7
- sessionId -> 8
- song title -> 9
- userId -> 10

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [28]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [29]:
# Create a Keyspace sparkify

session.execute(
    """
    CREATE KEYSPACE IF NOT EXISTs sparkify
    WITH REPLICATION = 
    {'class': 'SimpleStrategy', 'replication_factor': 1}
    """)

#### Set Keyspace

In [30]:
# Set KEYSPACE to the keyspace sparkify

session.set_keyspace('sparkify')


### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




### --------------------------------------------------------------------------------------------------------------------------------------------------------------

### Query 1: Give me the artist, song title and song's length in the music app history that was heard during sessionId = 338, and itemInSession = 4

In [31]:

## For Query 1, we need the result based on the specified sessionId and itemInSession, so I picked them as my Partition Key and Clustering column respectively.
## Rest other columns are there as we want to display those data in the result query.

session.execute(
    """
    CREATE TABLE IF NOT EXISTS session_song
    (sessionId int, itemInSession int, artist text, songTitle text, songLength float, PRIMARY KEY(sessionId, itemInSession))
    """
    )            

In [32]:
# Code to set up the CSV file. And populated data inside table with that csv file
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
##  INSERT statements into the `query` variable
        query = "INSERT INTO session_song (sessionId, itemInSession, artist, songTitle, songLength)"
        query = query + "VALUES (%s, %s, %s, %s, %s)"
        ## Assign which column element should be assigned for each column in the INSERT statement.
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5]) ))

#### Do a SELECT to verify that the data have been inserted into each table

In [33]:
## SELECT statement to verify the data was entered into the table

## Give me the artist, song title and song's length in the music app history that was heard during sessionId = 338, and itemInSession = 4

rows = session.execute(""" SELECT artist, songTitle, songLength FROM session_song WHERE sessionId = 338 AND itemInSession = 4 """)

for row in rows:
    print(row.artist, row.songtitle, row.songlength)
    

###  Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name)\
### for userid = 10, sessionid = 182

In [34]:

## For Query 2, we need the result based on the specified userId and itemInSession, so I picked them as my Partition Key. \
## Also,the result should be sorted by itemInSession, so I picked it as my Clustering column.
## Rest other columns are there as we want to display those data in the result query.

session.execute(
    """
    CREATE TABLE IF NOT EXISTS user_id_song
    (userId int, sessionId int, itemInSession int, artist text, songTitle text, firstName text, lastName text,
    PRIMARY KEY ((userId, sessionId), itemInSession))
    """)

file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO user_id_song (userId, sessionId, artist, songTitle, firstName, lastName, itemInSession)"
        query = query + "VALUES (%s, %s, %s, %s, %s, %s, %s)"
        session.execute(query, (int(line[10]), int(line[8]), line[0], line[9], line[1], line[4], int(line[3]) ))       

In [35]:
## SELECT statement to verify the data was entered into the table

## Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name)\
## for userid = 10, sessionid = 182


rows = session.execute(""" SELECT artist, songTitle, firstName, lastName FROM user_id_song WHERE userId = 10 AND sessionId = 182 """)

for row in rows:
    print(row.artist, row.songtitle, row.firstname, row.lastname)

### Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'


In [36]:

## For Query 3, we need the result based on the specified songTitle, so I picked it as my Partition Key. \
## But,the songTitle isn't alone enough for unique idetification, hence I picked userId alongside as my Clustering column.
## Rest other columns are there as we want to display those data in the result query.

session.execute(
    """
    CREATE TABLE IF NOT EXISTS app_history_song
    (songTitle text, firstName text, lastName text, userId int, 
    PRIMARY KEY(songTitle, userId))
    """)

file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO app_history_song (songTitle, firstName, lastName, userId)"
        query = query + "VALUES (%s, %s, %s, %s)"
        session.execute(query, (line[9], line[1], line[4], int(line[10])) )      

In [37]:
## SELECT statement to verify the data was entered into the table

## Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

rows = session.execute(""" SELECT firstName, lastName FROM app_history_song WHERE songTitle = 'All Hands Against His Own' """)

for row in rows:
    print(row.firstname, row.lastname)

Sara Johnson


### Drop the tables before closing out the sessions

In [39]:
session.execute(""" DROP TABLE session_song """)

session.execute(""" DROP TABLE user_id_song """)

session.execute(""" DROP TABLE app_history_song """)


### Close the session and cluster connection¶

In [40]:
session.shutdown()
cluster.shutdown()